In [9]:
%%capture
!pip install -q yfinance numpy pandas matplotlib scikit-learn PyPortfolioOpt scipy seaborn boto3 openai

In [2]:
# yf.download downloads stock price data for each ticker
# Adj close pulls adjusted closing prices, accounting for splits and dividends for the most accurate price
# pct_change() calculates the percentage change between each row in the dataframe (represents daily stock returns as percentage) 

import yfinance as yf
import pandas as pd

pd.set_option('display.max_rows', 50)

tickers = ['AAPL', 'JNJ', 'XOM', 'PG', 'JPM']
start_date = '2024-10-01'
end_date = '2024-11-01'

data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

returns = data.pct_change().dropna()

returns

[*********************100%***********************]  5 of 5 completed


Ticker,AAPL,JNJ,JPM,PG,XOM
Date,,,,,
2024-10-02,0.002520,-0.005062,0.001207,-0.006472,0.013258
2024-10-03,-0.004895,-0.004157,-0.009938,-0.010470,0.008723
2024-10-04,0.005007,-0.001309,0.035494,-0.007289,0.018355
2024-10-07,-0.022531,-0.004741,-0.001373,-0.010422,0.004326
2024-10-08,0.018404,0.001003,-0.000853,0.006223,-0.026561
2024-10-09,0.016698,0.006012,0.012669,0.006601,0.000410
2024-10-10,-0.002178,-0.000871,-0.002718,-0.001890,0.008600
2024-10-11,-0.006505,0.005919,0.044400,0.012666,0.003817
2024-10-14,0.016480,0.000867,-0.003644,0.008300,0.003802


In the cell above, I have created a dataframe that displays the percentage change per day for the month of October with respect to each of the five stocks I have selected.